In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = os.environ["GROQ_API_KEY"]

llm = ChatGroq(model="llama3-8b-8192")

In [4]:
from langchain_core.runnables import RunnableLambda, RunnableSequence
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

### Classifier Chain

In [85]:
classifier_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a customer service agent who is expert in classifying the customer questions in ecommerce store."),
        ("human", "Classify the question: {question} as one of the following:\n1. 'product_inquiry': If the customer is asking about specific product price, product details or product stock only.\n2. 'general_information': If the customer is asking about the store information, store location, store policies or anything that is general information about the store.\n3. 'others': If the question does not fall in any of the above categories and is like out of topic for a customer support service.")
    ]
)

In [86]:
classification_chain = classifier_template | llm | StrOutputParser()

In [87]:
classification_chain

ChatPromptTemplate(input_variables=['question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a customer service agent who is expert in classifying the customer questions in ecommerce store.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template="Classify the question: {question} as one of the following:\n1. 'product_inquiry': If the customer is asking about specific product price, product details or product stock only.\n2. 'general_information': If the customer is asking about the store information, store location, store policies or anything that is general information about the store.\n3. 'others': If the question does not fall in any of the above categories and is like out of topic for a customer support service."))])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B04D85DA00>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B04D

In [92]:
classification_chain.invoke({
    "question": "what is the delivery options"
})

'I would classify the question "what is the delivery options" as:\n\n2. \'general_information\'\n\nThe question is asking about the delivery options, which is a general piece of information about the store\'s shipping policies. This type of question is not specific to a particular product, but rather provides information that customers should know when making a purchase.'

### SQL Chain

In [9]:
from langchain_core.runnables import RunnablePassthrough

In [10]:
from langchain_community.utilities import SQLDatabase

db_path = "products.db"

db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

In [11]:
from langchain.chains import create_sql_query_chain

In [12]:
import re

def extract_query(text):
    # Define the regular expression to find the SQL query part
    pattern = r'SQLQuery:\s*(.*)'
    match = re.search(pattern, text)
    
    if match:
        return match.group(1)
    else:
        return None

In [13]:
from langchain_core.runnables import RunnableLambda

extract_sql_query = RunnableLambda(lambda x: extract_query(x))

In [14]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

write_query  = create_sql_query_chain(llm, db)

query_getter = write_query | extract_sql_query

In [15]:
query_getter

RunnableAssign(mapper={
  input: RunnableLambda(...),
  table_info: RunnableLambda(...)
})
| RunnableLambda(lambda x: {k: v for k, v in x.items() if k not in ('question', 'table_names_to_use')})
| PromptTemplate(input_variables=['input', 'table_info'], partial_variables={'top_k': '5'}, template='You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables below. B

In [110]:
query_getter.invoke({"question": "Do you sell poco?"})

'SELECT "name" FROM products WHERE "name" LIKE \'%poco%\';'

In [17]:
execute_query = QuerySQLDataBaseTool(db=db)

In [18]:
execute_query

QuerySQLDataBaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001B04E91CEF0>)

In [111]:
execute_query.invoke({"query": "SELECT * FROM products WHERE name LIKE '%poco%'"})

''

In [112]:

answer_prompt = PromptTemplate.from_template(
    """
    "You are customer service agent for an e-commerce store located in Nepal.The store sell electronics and gadgets and you are chatting with a customer who need help.
    Given the following user question, corresponding  SQL query, and the result of the query, respond to the user with the suitable answer to the question without adding anything up in a short and concise manner as possible.
    
    If the SQL Result is empty, dont try to make up answer, just negate the user question.
    
    Question: <question>{question}</question>
    SQL Query: <query>{query}</query>
    SQL Result: <result>{result}</result>
    Answer:
    """
)

answer_chain = answer_prompt | llm | StrOutputParser()

In [113]:
from operator import itemgetter

sql_chain = (
    RunnablePassthrough.assign(query=query_getter).assign(result=itemgetter("query") | execute_query)
    | answer_chain
)

In [114]:
sql_chain

RunnableAssign(mapper={
  query: RunnableAssign(mapper={
           input: RunnableLambda(...),
           table_info: RunnableLambda(...)
         })
         | RunnableLambda(lambda x: {k: v for k, v in x.items() if k not in ('question', 'table_names_to_use')})
         | PromptTemplate(input_variables=['input', 'table_info'], partial_variables={'top_k': '5'}, template='You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\

In [119]:
response = sql_chain.invoke({"question": "Do you have calculator in your store?"})

In [120]:
print(response)

Yes, we have calculators in our store.


In [24]:
response = sql_chain.invoke({"question": "What is the price of redmi note 9 pro"})

In [25]:
print(response)

The price of Redmi Note 9 Pro is NPR 20,000.


In [26]:
response = sql_chain.invoke({"question": "Do you have samsung galaxy s29 ultra in stock currently?"})

In [27]:
response

'Thank you for reaching out to us! According to our current stock, we do have Samsung Galaxy S29 Ultra in stock. You can place your order for it.'

### RAG CHAIN

In [28]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

C:\Users\acer\AppData\Roaming\Python\Python312\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [29]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [30]:
embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-MiniLM-L6-v2",
)

C:\Users\acer\AppData\Roaming\Python\Python312\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
C:\Users\acer\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [31]:
os.environ["PINECONE_API_KEY"] = os.environ["PINECONE_API_KEY"]
index_name = "storeagent"

vectorstore = PineconeVectorStore(
    index_name=index_name,
    embedding=embedding,
)

In [32]:
# from langchain.text_splitter import CharacterTextSplitter
# from langchain_community.document_loaders import TextLoader

# file_path = "info.txt"

# loader = TextLoader(file_path=file_path, encoding="utf-8")
# documents = loader.load()

# text_splitter = CharacterTextSplitter(
#     chunk_size=200,
#     chunk_overlap=0
# )
# docs = text_splitter.split_documents(documents=documents)

# vectorstore.add_documents(docs)

In [33]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k":2
    }
)
retriever.invoke("Delivery options")

[Document(page_content='## What are the delivery options\n- The delivery  options is available all over Nepal and the delivery charge is Rs. 120 standard price and delivery is free inside Dharan area.', metadata={'source': 'info.txt'}),
 Document(page_content='## How long the delivery takes\n- The delivery takes about 2-3 days for all over Nepal and if you are within Dharan Delivery will be the same day you order product.', metadata={'source': 'info.txt'})]

In [34]:
chat_system_prompt = (
    "You are customer service agent for an e-commerce store."
    "The store sell electronics and gadgets and you are chatting with a customer who need help."
    "Use the retrieved information or chat history to answer the customer's question."
    "If the retrieved information is not useful or the chat history is not relevant, you can ignore it and just answer the question."
    "but remember you only answer about the store and questions related to it and nothing else."
    "If asked anything else just say that you are just customer service agent and you can only answer about the store."
    "\n\n"
    "{context}"
)

In [41]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", chat_system_prompt),
        ("human", "{input}"),
    ]
)

In [42]:
chat_chain = create_stuff_documents_chain(
    llm,
    chat_prompt
)

In [43]:
rag_chain = create_retrieval_chain(
    retriever, # retrieve documents
    chat_chain # get the retrieved documents and pass it to LLM to answer the question
)

In [44]:
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['PineconeVectorStore', 'HuggingFaceEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x000001B04D8C8F50>, search_kwargs={'k': 2}), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context', 'input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="You are customer service agent for an e-commerce store.The store sell electronics and gadgets and you are chatting with a customer who need help.Use the retrieved information or chat history to answer the customer's question.If the retrieved infor

In [46]:
response = rag_chain.invoke({"input": "What are the delivery options?"})

In [47]:
response["answer"]

'We offer delivery options all over Nepal. The standard delivery charge is Rs. 120. However, if you are within the Dharan area, your order will be delivered free of charge.'

### Conditional Chain

In [48]:
handle_others_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a customer service representative for an e-commerce store. You are chatting with a customer who needs help."),
        ("human", "You received a question which is not related to you so you should not have to answer it so just say that you are a customer service agent and you can only answer about the store.")
    ]
)

## APPROACH1: Runnable Branch

In [93]:
def getDict(question):
    print(question)
    return {"question": question}
def getDictRag(question):
    print(question)
    return {"input": question}

In [133]:
from langchain_core.runnables import RunnableBranch

branches = RunnableBranch(
    (
        lambda x: "product_inquiry" in x, # condition
        RunnableLambda(lambda x: getDict(x)) | sql_chain
    ),
    (
        lambda x: "general_information" in x, # condition
        # rag_chain,
        RunnableLambda(lambda x: getDictRag(x)) | rag_chain | RunnableLambda(lambda x: x["answer"])
    ),
    handle_others_template | llm | StrOutputParser()
)

In [134]:
sql_rag_chain = classification_chain | branches

In [52]:
sql_rag_chain

ChatPromptTemplate(input_variables=['question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a customer service agent who is expert in classifying the customer questions in ecommerce store.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template="Classify the question: {question} as one of the following:\n1. 'product_inquiry': If the customer is asking about product details, product availability, product inquiry or anything about the products that require looking up to database.\n2. 'general_information': If the customer is asking about the store information, store location, store policies or anything that is general information about the store.\n3. 'others': If the question does not fall in any of the above categories and is like out of topic for a customer support service."))])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B04D85DA00>, async_client=<groq.resour

In [53]:
response = sql_rag_chain.invoke({"question": "Where is the store located?"})

I would classify the question "Where is the store located?" as:

2. 'general_information'

This is because the customer is asking about the store's physical location, which is a piece of general information about the store and not related to a specific product or product inquiry.


In [54]:
response

{'input': 'I would classify the question "Where is the store located?" as:\n\n2. \'general_information\'\n\nThis is because the customer is asking about the store\'s physical location, which is a piece of general information about the store and not related to a specific product or product inquiry.',
 'context': [Document(page_content='## What is the name of the store?\n- The name of the store is All Electronics store.', metadata={'source': 'info.txt'}),
  Document(page_content='## Where is the store located?\n- The Store is located in Dharan sunsari district Nepal.\n\n## Who own the store?\n- The owner is Manoj Baniya.', metadata={'source': 'info.txt'})],
 'answer': "Thank you for the clarification!\n\nYou're welcome to ask me anything about All Electronics Store, and I'll do my best to help."}

## APPROACH2: RUNNABLE LAMBDA CUSTOM FUNCTION

In [121]:
general_chain = PromptTemplate.from_template(
    """
    Respond to the user that you dont answer that question as you are a customer service agent and you can only answer about the store.
    """
) | llm | StrOutputParser()

In [135]:
def route(info):
    print(info)
    if "general_information" in info["topic"].lower():
        question = info["question"]
        return RunnableLambda(lambda x: {"input": question}) | rag_chain | RunnableLambda(lambda x: x["answer"])
    elif "product_inquiry" in info["topic"].lower():
        return sql_chain
    else:
        return general_chain

In [136]:
sql_rag_chain_2 = {"topic": classification_chain, "question": lambda x: x["question"]} | RunnableLambda(
    route
)

In [124]:
sql_rag_chain_2

{
  topic: ChatPromptTemplate(input_variables=['question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a customer service agent who is expert in classifying the customer questions in ecommerce store.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template="Classify the question: {question} as one of the following:\n1. 'product_inquiry': If the customer is asking about specific product price, product details or product stock only.\n2. 'general_information': If the customer is asking about the store information, store location, store policies or anything that is general information about the store.\n3. 'others': If the question does not fall in any of the above categories and is like out of topic for a customer support service."))])
         | ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B04D85DA00>, async_client=<groq.resources.chat.completions.AsyncCompletions ob

In [80]:
sql_rag_chain_2.invoke({"question": "what is the price of redmi note 9 pro"})

{'topic': 'Based on the question "what is the price of redmi note 9 pro", I would classify it as:\n\n1. \'product_inquiry\'\n\nThe customer is asking about the price of a specific product, which requires looking up the database to provide the information.', 'question': 'what is the price of redmi note 9 pro'}


'Dear customer, the price of Redmi Note 9 Pro is NPR 20,000.'

In [81]:
sql_rag_chain_2.invoke({"question": "what is 2 + 2"})

{'topic': 'I would classify the question "what is 2 + 2" as \'others\'. This question is not related to the ecommerce store or its products, and is a basic math question that is outside the scope of the customer support service.', 'question': 'what is 2 + 2'}


"I'm happy to help you with your inquiry! However, I want to clarify that I'm a customer service agent for our store, and I'm only authorized to provide information and assistance related to our products, services, and policies. I apologize, but I won't be able to answer questions that fall outside of our store's scope.\n\nIf you have any questions or concerns about our store, such as product availability, pricing, or return policies, I'd be more than happy to help. Please feel free to ask me anything related to our store, and I'll do my best to assist you."

In [137]:
sql_rag_chain_2.invoke({"question": "where is the store located?"})

{'topic': 'I would classify the question "where is the store located?" as:\n\n2. \'general_information\'\n\nThis question falls under the category of general information about the store, as the customer is asking about the physical location of the store.', 'question': 'where is the store located?'}


'The All Electronics store is located in Dharan, Sunsari district, Nepal.'

In [132]:
sql_rag_chain_2.invoke({"question": "What is the delivery options?"})

{'topic': 'I would classify the question "What is the delivery options?" as:\n\n2. \'general_information\'\n\nThis question is asking about the store\'s delivery options, which is general information about the store\'s policies and services. It\'s not specifically asking about a product, so it doesn\'t fit into the \'product_inquiry\' category.', 'question': 'What is the delivery options?'}


{'input': 'What is the delivery options?',
 'context': [Document(page_content='## What are the delivery options\n- The delivery  options is available all over Nepal and the delivery charge is Rs. 120 standard price and delivery is free inside Dharan area.', metadata={'source': 'info.txt'}),
  Document(page_content='## How long the delivery takes\n- The delivery takes about 2-3 days for all over Nepal and if you are within Dharan Delivery will be the same day you order product.', metadata={'source': 'info.txt'})],
 'answer': "Thank you for reaching out! According to our store's delivery policy, we have delivery options available all over Nepal. The delivery charge is Rs. 120, which is our standard price. However, if you are within the Dharan area, your delivery is absolutely free!"}

In [126]:
sql_rag_chain_2.invoke({"question": "How many products are there?"})

{'topic': 'I would classify the question "How many products are there?" as \'product_inquiry\'. The customer is asking about the number of products available in the store, which falls under the category of product information and is a specific inquiry about a product.', 'question': 'How many products are there?'}


'We have a total of 9 products available in our store.'

In [129]:
sql_rag_chain_2.invoke({"question": "DO you sell playstation?"})

{'topic': 'I would classify the question "DO you sell playstation?" as \'product_inquiry\'. The customer is asking if the store sells a specific product (PlayStation), which falls under the category of product inquiry.', 'question': 'DO you sell playstation?'}


"I'm happy to help! We don't currently have playstation products in our inventory."